functions used for preprocessing and cleaning up

In [20]:
def crp(img):
    ht, wd= img.shape
    ww = 500
    hh = 500
    result = np.full((hh,ww), 0, dtype=np.uint8)
    xx = (ww - wd) // 2
    yy = (hh - ht) // 2
    result[yy:yy+ht, xx:xx+wd] = img
    final =cv2.resize(result, (50,50))
    return final

In [21]:
import cv2
def countor(input):
    img=cv2.imread(input,cv2.IMREAD_GRAYSCALE)
    thres=cv2.adaptiveThreshold(img, 255.0, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 51, -20.0)
    kernel = np.ones((5,5),np.uint8)
    out = cv2.erode(thres, kernel)
    c,h=cv2.findContours(out, cv2.RETR_EXTERNAL,  cv2.CHAIN_APPROX_SIMPLE)
    print("Number of items detected: ",len(c))
    return c , thres  

creating the dierctory and cleaning up image
we only train with broken and full rice images, that is mixed rice is not used

In [23]:
import os
from os import listdir
import numpy as np
for image in  os.listdir():
    filename = os.fsdecode(image)
    print(filename)
    if filename.startswith("broken_"):
        if not os.path.exists("broken/broken_rice"):
           os.makedirs("broken/broken_rice")
        path='broken/broken_rice'
        c,thres=countor(image)
        nn=1
        for i in c:
            x, y, width, height = cv2.boundingRect(i)
            cimg = thres[y:y+height, x:x+width]
            cimg=crp(cimg)
            cv2.imwrite(path+str(nn)+".png",cimg)
            nn=nn+1
    elif filename.startswith("full_"):
        if not os.path.exists("full/full_rice"):
           os.makedirs("full/full_rice")
        path='full/full_rice'
        c,thres=countor(image)
        nn=1
        for i in c:
            x, y, width, height = cv2.boundingRect(i)
            cimg = thres[y:y+height, x:x+width]
            cimg=crp(cimg)
            cv2.imwrite(path+str(nn)+".png",cimg)
            nn=nn+1
    else:
        pass

broken
broken_grain_1.jpg
Number of items detected:  650
broken_grain_2.jpg
Number of items detected:  526
broken_grain_3.jpg
Number of items detected:  937
full
full_grain_1.jpg
Number of items detected:  300
full_grain_2.jpg
Number of items detected:  493
full_grain_3.jpg
Number of items detected:  680
full_grain_4.jpg
Number of items detected:  613
full_grain_5.jpg
Number of items detected:  569
full_grain_6.jpg
Number of items detected:  619
mixed_grains_3.jpg
mixed_grain_1.jpg
mixed_grain_2.jpg


this will helps to split the labelled preprocessed images into train and validation set

In [27]:
import splitfolders
splitfolders.ratio("train", output="full_split", seed=1337, ratio=(.8,.2)) 

In [43]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1. / 255)
val_datagen=ImageDataGenerator(rescale=1. / 255)
train_data= train_datagen.flow_from_directory(
    directory=r"Downloads\data\data\full_split\train",
    color_mode='grayscale',
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    seed=42
)
val_data= val_datagen.flow_from_directory(
    directory=r"Downloads\data\data\full_split\val",
    color_mode='grayscale',
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    seed=42
)

Found 1293 images belonging to 2 classes.
Found 324 images belonging to 2 classes.


In [46]:
a, b = next(train_data)
a.shape

(32, 224, 224, 1)

In [47]:
import keras
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import optimizers
from keras.layers import Dropout
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(224,224,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=keras.optimizers.Adam(lr=.0001),
              metrics=['accuracy'])

C:\Users\shaha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [48]:
import sys
import numpy
from PIL import Image
sys.modules['Image'] = Image 
md = model.fit_generator(
    train_data,
    steps_per_epoch=10,
    epochs=10,
    validation_data=val_data,
    validation_steps=2)

Epoch 1/10


C:\Users\shaha\AppData\Local\Temp\ipykernel_20308\993764796.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  md = model.fit_generator(


10/10 [==============================] - 25s 2s/step - loss: 0.6351 - accuracy: 0.6656 - val_loss: 0.5094 - val_accuracy: 0.7969
Epoch 2/10
10/10 [==============================] - 21s 2s/step - loss: 0.5495 - accuracy: 0.7437 - val_loss: 0.3550 - val_accuracy: 0.9219
Epoch 3/10
10/10 [==============================] - 20s 2s/step - loss: 0.4685 - accuracy: 0.7907 - val_loss: 0.3276 - val_accuracy: 0.8750
Epoch 4/10
10/10 [==============================] - 21s 2s/step - loss: 0.4278 - accuracy: 0.8344 - val_loss: 0.5655 - val_accuracy: 0.7188
Epoch 5/10
10/10 [==============================] - 20s 2s/step - loss: 0.3860 - accuracy: 0.8250 - val_loss: 0.5251 - val_accuracy: 0.7188
Epoch 6/10
10/10 [==============================] - 21s 2s/step - loss: 0.4508 - accuracy: 0.8040 - val_loss: 0.4591 - val_accuracy: 0.7812
Epoch 7/10
10/10 [==============================] - 20s 2s/step - loss: 0.4227 - accuracy: 0.8272 - val_loss: 0.4583 - val_accuracy: 0.7969
Epoch 8/10
10/10 [=============

testing with the 5 images given

In [53]:
import os
from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array 
import numpy as np
fold=1
for img in  os.listdir():
    filename = os.fsdecode(img)
    print(filename)
    if filename.startswith("ima"):
       if not os.path.exists('test/'+str(fold)):
           os.makedirs('test/'+str(fold))
       path='test/'+str(fold)+'/'
       c,thres=countor(img)
       nn=1
       for i in c:
            x, y, width, height = cv2.boundingRect(i)
            cimg = thres[y:y+height, x:x+width]
            cimg=crp
            (cimg)
            cv2.imwrite(path+str(nn)+".png",cimg)
            nn=nn+1
    fold=fold+1
    

desktop.ini
image_1.jpg
Number of items detected:  585
image_2.jpg
Number of items detected:  595
image_3.jpg
Number of items detected:  595
image_4.jpg
Number of items detected:  663
image_5.jpg
Number of items detected:  705
test


In [ ]:
from tensorflow.keras.preprocessing import image
for fil in  os.listdir():
   broken=0
   full=0
   for f in os.listdir(fil):
        img = image.load_img(fil+'/'+f, target_size=(224,224), grayscale=True)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])
        classes = model.predict(images)
        classes=np.where(classes > 0, 1,0)
        if classes == 0:
           broken += 1
        else:
           full += 1
   print("Total full :",full)
   print("Total broken :",broken)
   print('Total : ',full+broken)


